In [ ]:
# Este es el modelo de clasificación para la Estrategia 2. Clasifica los vehículos como premium >= 70.000 euros y no premium < 70.000 euros

In [1]:
# Paso 1: Importar las bibliotecas necesarias

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE


In [2]:
# Paso 2: Carga de datos

df = pd.read_csv("../data/feature_engineering.csv")

In [3]:
df70 = df.copy()
df70["Caro"] = df70["price"] >= 70000
df70.drop(columns="price", inplace=True)

In [4]:
df70

,year,kms,power,make_Abarth,make_Alfa Romeo,make_Alpine,make_Aston Martin,make_Audi,make_BMW,make_Bentley,...,segmento_B,segmento_C,segmento_D,segmento_E,segmento_F,segmento_J,segmento_M,segmento_S,segmento_otros,Caro
0,2017,93000.0,110.0,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,2019,151716.0,131.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,2018,84490.0,85.0,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,2018,172844.0,286.0,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
4,2015,69000.0,130.0,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8943,2013,137186.0,211.0,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
8944,2014,137000.0,258.0,False,False,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
8945,2016,29445.0,90.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
8946,2003,111980.0,395.0,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [5]:
# Separar características (X) y variable objetivo (y)
X = df70.drop(columns="Caro")
y = df70["Caro"]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Random Forest Classifier

In [5]:
# Aplicar SMOTE al train
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

In [6]:

# Definir el modelo base
rf_cls_model = RandomForestClassifier(random_state=42)

# Espacio de búsqueda de hiperparámetros
param_grid = {
    "n_estimators": [10, 100, 1000],
    "max_features": [1,2,3]
}

# Configurar búsqueda en cuadrícula con validación cruzada
grid_search_rf_clas = GridSearchCV(rf_cls_model,
                         param_grid,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

# Entrenar el modelo con el conjunto de entrenamiento
grid_search_rf_clas.fit(X_train_bal, y_train_bal)

# Mejor modelo encontrado
best_rf_clas = grid_search_rf_clas.best_estimator_

# Predicciones
y_test_pred = best_rf_clas.predict(X_test)

print(grid_search_rf_clas.best_score_)
print(grid_search_rf_clas.best_params_)
print(grid_search_rf_clas.best_estimator_)

# Resultados
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_test_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_test_pred))




Fitting 10 folds for each of 9 candidates, totalling 90 fits
0.9944634314115925
{'max_features': 1, 'n_estimators': 1000}
RandomForestClassifier(max_features=1, n_estimators=1000, random_state=42)
Matriz de confusión:
[[1734    5]
 [  14   37]]

Reporte de clasificación:
              precision    recall  f1-score   support

       False       0.99      1.00      0.99      1739
        True       0.88      0.73      0.80        51

    accuracy                           0.99      1790
   macro avg       0.94      0.86      0.90      1790
weighted avg       0.99      0.99      0.99      1790



In [7]:
# Vamos a guardar un modelo

from joblib import dump

# Guardar el modelo para clasificar los vehículos entre caros (o premium) de más de 70000 euros y baratos (o no premium) de menos de 70000 euros. En este caso un Random Forest
dump(best_rf_clas, "../model/Modelo2_clasificacion_rf.pkl")
print("✅ Modelo de Random Forest de clasificacion guardado como 'Modelo2_clasificacion_rf.pkl'")


✅ Modelo de Random Forest de clasificacion guardado como 'Modelo2_clasificacion_rf.pkl'
